In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
np.random.seed(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer import Trainer
from src.Focal_loss import FocalLoss
from src.Model_class import Model_class
from src.Loss_class import Loss_class
from src.DatasetClass import TableDatasetDF
from src.EncDataset import EncoderDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 1024
INIT_PARAM = 256
NUM_FEATURES = 60

In [3]:
df = pd.read_csv('../data/data_fe.csv')
feature_importance = pd.read_csv('../data/feature_imporstance_WS.csv')

In [4]:
features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

df.to_csv('../data/df_to_enc.csv', index = False)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'], random_state=42)

In [6]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis = 1)

train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [7]:
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [8]:
class MyModel(nn.Module):
    def __init__(self, num_features, init_param):
        super(MyModel, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(num_features, init_param),
            nn.BatchNorm1d(init_param),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Linear(init_param, int(init_param/2)),
            nn.BatchNorm1d(int(init_param/2)),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer3 = nn.Sequential(
            nn.Linear(int(init_param/2), int(init_param/4)),
            nn.BatchNorm1d(int(init_param/4)),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(
            nn.Linear(int(init_param/4), int(init_param/8)),
            nn.BatchNorm1d(int(init_param/8)),
            nn.Dropout(0.1),
            nn.ReLU()
        )

        self.layer5 = nn.Linear(int(init_param/8), 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

In [9]:
# layers = [
#     nn.Linear(NUM_FEATURES, INIT_PARAM),
#     nn.BatchNorm1d(INIT_PARAM),
#     nn.Dropout(0.1),
#     nn.ReLU(),

#     nn.Linear(INIT_PARAM, int(INIT_PARAM/2)),
#     nn.BatchNorm1d(int(INIT_PARAM/2)),
#     nn.Dropout(0.1),
#     nn.ReLU(),

#     nn.Linear(int(INIT_PARAM/2), int(INIT_PARAM/4)),
#     nn.BatchNorm1d(int(INIT_PARAM/4)),
#     nn.Dropout(0.1),
#     nn.ReLU(),

#     nn.Linear(int(INIT_PARAM/4), int(INIT_PARAM/8)),
#     nn.BatchNorm1d(int(INIT_PARAM/8)),
#     nn.Dropout(0.1),
#     nn.ReLU(),
    
#     nn.Linear(int(INIT_PARAM/8), 2)
# ]

# model = nn.Sequential(*layers)


model = MyModel(NUM_FEATURES, INIT_PARAM)


In [10]:
loss = Loss_class(FocalLoss(gamma=2))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'fe_{INIT_PARAM}_{BATCH_SIZE}_{NUM_FEATURES}_TEST_MODEL',
    project="Internship_project",
    dir = '../logs/'
)

In [11]:
trainer = Trainer(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/',
                  saving_model=True
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [12]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 53.72it/s]


Epoch: 1 of 40, 0.034 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 50.27it/s]


Epoch: 2 of 40, 0.009 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 53.71it/s]


...

Epoch: 38 of 40, 0.010 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 50.13it/s]


Epoch: 39 of 40, 0.009 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 47.75it/s]


Epoch: 40 of 40, 0.009 min
